In [1]:
import gym
import tensorflow as tf
import numpy as np

In [2]:
env = gym.make("CartPole-v0")

In [3]:
obs_dim = env.observation_space.shape[0]
n_acts = env.action_space.n
print("obs = {}, n_acts = {}".format(obs_dim, n_acts))

obs = 4, n_acts = 2


In [4]:
obs = env.reset()
obs

array([-0.00114128, -0.01948678, -0.00073424,  0.04375321])

In [5]:
def mpl(x, sizes, activation = tf.tanh, output_activation=None):

    for size in sizes:
        x = tf.layers.dense(x,units = size, activation = activation)
    
    return tf.layers.dense(x, units = sizes[-1], activation = output_activation)


In [6]:
#first layer of NN
obs_ph = tf.placeholder(shape=(None, obs_dim), dtype=tf.float32)
#creating NN
logit = mpl(obs_ph, sizes = [32,64]+[n_acts])
#array([[0.01685937, 0.00622761]], dtype=float32)

Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [7]:
max_action = tf.multinomial(logits=logit,num_samples=1)
#array([[0]])
action = tf.squeeze(max_action, axis=1)
#array([0])

Instructions for updating:
Use `tf.random.categorical` instead.


In [8]:
#loss
weights_ph = tf.placeholder(shape=(None,), dtype = tf.float32)
act_ph = tf.placeholder(shape=(None,), dtype = tf.int32)

#one_hot taken actions
action_masks = tf.one_hot(act_ph, n_acts)
#array([[0., 1.]...], dtype=float32)

#reward that will get from this point
baseline_ph = tf.placeholder(shape=(None,), dtype = tf.float32)

In [9]:
baseline = baseline_ph - 1

In [10]:
#logprob(a|s)
log_probs = tf.reduce_sum(action_masks * tf.nn.log_softmax(logit), axis=1)
#array([-0.6952652,...], dtype=float32)
loss = -tf.reduce_mean(weights_ph* log_probs* baseline)
#9.812662

In [11]:
#optimizer
train_op = tf.train.AdamOptimizer(learning_rate = 1e-2).minimize(loss)
#loss minimizes :D

In [12]:
#variables for initializing and using
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

In [13]:
max_action = sess.run(action, {obs_ph: obs.reshape(1,-1)})[0]
#get element from array 0 or 1

In [14]:
def reward_to_go(rews):
    n = len(rews)
    rtgs = np.zeros_like(rews)
    for i in reversed(range(n)):
        rtgs[i] = rews[i] + (rtgs[i+1] if i+1 < n else 0)
    #returns [200. 199. ... 1.]
    return rtgs

In [15]:
#LMAO I DONT KNOW HOW IT WORKS, BUT IT WORKS
def reward_after(w):
    n = len(w)
    list2 = w.copy()
    for i in range(n):
        prev_el = w[i]
        su = 0
        for j in reversed(range(i)):
            if prev_el >= w[j]:
                break
            su+=1
            prev_el = w[j]
        
        list2[i]=su if su != 0 else 1
    return list2
#w[i]+=j WORK
#w[i]=200-w[i] NOT WORKING

In [16]:
a = []
b = []
for i in reversed(range(200)):
    a.append(i)
a = np.array(a)
a = a.reshape(len(a))
for i in reversed(range(100)):
    b.append(i)
b = np.array(b)
b = b.reshape(len(b))
ab = np.concatenate((a,b),axis=0)
print("ab-conc;:::\n",ab)

reward_after(ab)

ab-conc;:::
 [199 198 197 196 195 194 193 192 191 190 189 188 187 186 185 184 183 182
 181 180 179 178 177 176 175 174 173 172 171 170 169 168 167 166 165 164
 163 162 161 160 159 158 157 156 155 154 153 152 151 150 149 148 147 146
 145 144 143 142 141 140 139 138 137 136 135 134 133 132 131 130 129 128
 127 126 125 124 123 122 121 120 119 118 117 116 115 114 113 112 111 110
 109 108 107 106 105 104 103 102 101 100  99  98  97  96  95  94  93  92
  91  90  89  88  87  86  85  84  83  82  81  80  79  78  77  76  75  74
  73  72  71  70  69  68  67  66  65  64  63  62  61  60  59  58  57  56
  55  54  53  52  51  50  49  48  47  46  45  44  43  42  41  40  39  38
  37  36  35  34  33  32  31  30  29  28  27  26  25  24  23  22  21  20
  19  18  17  16  15  14  13  12  11  10   9   8   7   6   5   4   3   2
   1   0  99  98  97  96  95  94  93  92  91  90  89  88  87  86  85  84
  83  82  81  80  79  78  77  76  75  74  73  72  71  70  69  68  67  66
  65  64  63  62  61  60  59  58  57  

array([  1,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
       156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
       169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 18

In [18]:
def train_one_epoch(batch_size = 5000):
    
    batch_obs = []
    batch_acts = []
    batch_rets = []
    batch_lens = []
    batch_weights = [] #[200.0, 199.0, ... 1.0, 200.0, ... 1.0]
    obs = env.reset()
    done = False
    ep_rews = []
    rendering_epoch = True
    
    while True:
        
        if rendering_epoch == True:
            env.render() 
        batch_obs.append(obs.copy())
        
        act = sess.run(action, {obs_ph: obs.reshape(1,-1)})[0]
        
        obs, reward, done, _ = env.step(act)
        
        batch_acts.append(act)
        ep_rews.append(reward)
        
        if done:
            rendering_epoch = False
            #recording everything!
            ep_ret, ep_len = sum(ep_rews), len(ep_rews)
            
            batch_rets.append(ep_ret)
            batch_lens.append(ep_len)
            
            # the weight for each logprob(a_t|s_t) is reward-to-go from t
            batch_weights += list(reward_to_go(ep_rews))
            
            #reset vatiables
            obs, done, ep_rews = env.reset(), False, []
            
            if len(batch_obs)>batch_size:
                break
    if np.mean(batch_lens)<200:
        baseline = reward_after(batch_weights)
        batch_loss, _ = sess.run([loss, train_op], {
            obs_ph: batch_obs,
            act_ph: batch_acts,
            weights_ph: batch_weights,
            baseline_ph: baseline
        })
    else:
        batch_loss = 0
    return batch_loss, batch_rets, batch_lens

In [19]:
for i in range(500):
    batch_loss, batch_rets, batch_lens = train_one_epoch()
    print("#%i, batch_loss: %.3f, batch_rets: %.3f, batch_lens: %.3f" \
          %(i, batch_loss, np.mean(batch_rets), np.mean(batch_lens)))
    if batch_loss == 0:
        break


#0, batch_loss: 108.468, batch_rets: 22.647, batch_lens: 22.647
#1, batch_loss: 202.407, batch_rets: 31.516, batch_lens: 31.516
#2, batch_loss: 307.435, batch_rets: 39.117, batch_lens: 39.117
#3, batch_loss: 359.817, batch_rets: 43.164, batch_lens: 43.164
#4, batch_loss: 825.101, batch_rets: 56.944, batch_lens: 56.944
#5, batch_loss: 950.259, batch_rets: 66.303, batch_lens: 66.303
#6, batch_loss: 794.350, batch_rets: 60.578, batch_lens: 60.578
#7, batch_loss: 1349.886, batch_rets: 78.781, batch_lens: 78.781
#8, batch_loss: 1702.514, batch_rets: 80.968, batch_lens: 80.968
#9, batch_loss: 1887.283, batch_rets: 89.035, batch_lens: 89.035
#10, batch_loss: 1844.193, batch_rets: 94.528, batch_lens: 94.528
#11, batch_loss: 2387.523, batch_rets: 117.047, batch_lens: 117.047
#12, batch_loss: 2417.651, batch_rets: 114.622, batch_lens: 114.622
#13, batch_loss: 2952.917, batch_rets: 135.541, batch_lens: 135.541
#14, batch_loss: 2597.264, batch_rets: 139.333, batch_lens: 139.333
#15, batch_loss: 28

In [ ]:
def play_episode():
    
    rewards = 0
    obs = env.reset()
    while True:
        env.render()
        act = sess.run(action, {obs_ph: obs.reshape(1,-1)})[0]
        obs, reward, is_done, _ = env.step(act)
        rewards += reward
        #if is_done or rewards >200:
         #   break
    return rewards    

In [ ]:
print(play_episode())

In [ ]:
env.close()

In [ ]:
sum = 8
for i in range(8):
    print(i)
    sum+=i
sum